In [1]:
# importing required packages

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statistics import mean
from statistics import median

# Data Cleaning

In [2]:
"""Read data. This is the file you export from the Plan A Platform, listing each input field as one row. For example, 
the field "Waste from operations - residual weight in kg" in January 2021 would result in one row in the data export.
"""

# Loading the raw data. 
#Adjust the title of your CSV export file accordingly. Make sure it's in the same location as your Jupyter Notebook.
data = pd.read_csv("Green Team.csv")

# Filter on the year, you want to look at, for example 2021.
data = data[data['date(year-month)'].str.contains("2021")]

#replace no-data, and no by null values. "No-data" is the default value when nothing has been entered into a field. 
# "no" is what is recorded when a checkbox (i.e. asking whether national averages should be used) is NOT ticked. 
# we want to treat "no-data" and "no" as non-existing or null values for our calculations. 
data = data.replace('no-data', np.nan)
data = data.replace('no', np.nan)

#remove unnecessary columns
data = data.drop(columns=['help'])

# national averages are not available for vehicle fleet, waste, hosted and cloud servers. 
# The only option to tick is "We don't use ..."

data

,date(year-month),facility,input_type,value
336,2021-01,89-Headquarters,Business travel - short-distance car ride <25k...,NaN
337,2021-01,89-Headquarters,Business travel - long-distance car ride >25km...,NaN
338,2021-01,89-Headquarters,Business travel - short-distance train ride <2...,NaN
339,2021-01,89-Headquarters,Business travel - long-distance train ride >20...,NaN
340,2021-01,89-Headquarters,Business travel - short-distance flight <1500k...,NaN
...,...,...,...,...
1759,2021-12,364-Paris Office,Waste from operations - paper weight in kg,NaN
1760,2021-12,364-Paris Office,Waste from operations - residual weight in kg,NaN
1761,2021-12,364-Paris Office,Waste from operations - organic weight in kg,NaN
1762,2021-12,364-Paris Office,Waste from operations - glass weight in kg,NaN


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1008 entries, 336 to 1763
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   date(year-month)  1008 non-null   object
 1   facility          1008 non-null   object
 2   input_type        1008 non-null   object
 3   value             226 non-null    object
dtypes: object(4)
memory usage: 39.4+ KB


# Calculate Data Coverage and Data Gaps
The idea is to calculate the data coverage and data gaps respectively for each emissions category and each facility  per quarter. This should give an indication where some adjustments to the data collection process are necessary. 

Data Coverage: "To which extent was data provided for the given category in the given month/quarter?"

Data Gap: "How much of the data we'd need to provide for category X in quarter Y is missing?"

In [4]:
# create a list of facilities that are applicable to the data set
facilities = list(data.facility.unique())

# create a list of months that we need to look at individually
months = list(data['date(year-month)'].unique())
quarters = ["-01|-02|-03", "-04|-05|-06", "-07|-08|-09", "-10|-11|-12"]

# add the categories you want to look at to the list
categories = ['Business travel', 'Energy', 'commute', 'Facility', 'Hosted', 'Cloud', 'Vehicle', 'Waste']

In [5]:
# Set the timeframe you want to look at: either "quarters" or "months"
timeframe = quarters.copy()

In [6]:
# In this case, we use a counter because we want to disregard Q1 - Q3 when it comes to the data gaps and coverages of 
# the Paris office

# Let's create a data frame as a result listing the Data Coverages and Gaps for each quarter and each facility. 

quarter_r = []
facility_r = []
category_r = []
data_coverage = []
data_gap = []

counter_t = 1

for t in timeframe: 
    for c in categories:
        for f in facilities:
            # The Paris Office was opened in September 2021 only, so only Q4 is relevant (in September it was Laurent only)
            if (counter_t < 4) and (f == "364-Paris Office"):
                break
            else: 
                #print (str (t) + " " + str(c)+ " " + str(f))
                
                # Filter the clean data set on the particular category, facility and timeframe we're looking at in this cycle. 
                basis = data[data['input_type'].str.contains(c)]
                basis = basis[basis['facility'].str.contains(f)]
                basis = basis[basis['date(year-month)'].str.contains(t)]
                
                # At this point, we're looking at one particular emissions category, one facility and one
                # particular timeframe (month or quarter) only. Now, we want to count for each month, how many times
                # a non-null value is present. 
                helper = basis.value.notnull().groupby([basis['date(year-month)']]).sum().astype(int).reset_index(name='count')
                
                # if we look at quarters, what we get at this point is a table, listing each month of the quarter in
                # one column and another column called "count" that displays the number of times a non-null value
                # has been counted. 
                
                # To calculate the coverage (for the quarter), we count for how many months in this quarter data is
                # present, that is the "count"-value is greater than 0. 
                coverage = round(100*(helper['count'] > 0).sum() / len(helper))
                gap = 100 - coverage
                
                # Add the results of the current cycle to the respective lists
                quarter_r.append(t)
                facility_r.append(f)
                category_r.append(c)
                data_coverage.append(coverage / 100)
                data_gap.append(gap / 100)
                
    # The counter needs to be increased by 1 after each cycle.             
    counter_t += 1

result_1 = pd.DataFrame(list(zip(quarter_r, facility_r, category_r, data_coverage, data_gap)), columns = ['Quarter', 'Facility', 'Category', 'Data Coverage', 'Data Gap'])

# optionally, save result of data gaps and data coverages to CSV file
#result_1.to_csv("Data Coverages and Gaps Plan A 2021.csv")

result_1

,Quarter,Facility,Category,Data Coverage,Data Gap
0,-01|-02|-03,89-Headquarters,Business travel,0.67,0.33
1,-01|-02|-03,89-Headquarters,Energy,1.00,0.00
2,-01|-02|-03,89-Headquarters,commute,0.67,0.33
3,-01|-02|-03,89-Headquarters,Facility,1.00,0.00
4,-01|-02|-03,89-Headquarters,Hosted,0.00,1.00
5,-01|-02|-03,89-Headquarters,Cloud,0.00,1.00
6,-01|-02|-03,89-Headquarters,Vehicle,0.00,1.00
7,-01|-02|-03,89-Headquarters,Waste,0.00,1.00
8,-04|-05|-06,89-Headquarters,Business travel,1.00,0.00
9,-04|-05|-06,89-Headquarters,Energy,1.00,0.00


## Aggregate Data Coverages and Data Gaps

In [7]:
# aggregate 2021 data gaps and coverages per categories and per facility. Use the simple average to group.  
category_agg = result_1.groupby(['Facility', 'Category']).mean()
category_agg

Data Coverage  Data Gap
Facility         Category                                
364-Paris Office Business travel         0.0000    1.0000
                 Cloud                   0.0000    1.0000
                 Energy                  0.0000    1.0000
                 Facility                1.0000    0.0000
                 Hosted                  0.0000    1.0000
                 Vehicle                 0.0000    1.0000
                 Waste                   0.0000    1.0000
                 commute                 0.0000    1.0000
89-Headquarters  Business travel         0.9175    0.0825
                 Cloud                   0.4175    0.5825
                 Energy                  1.0000    0.0000
                 Facility                1.0000    0.0000
                 Hosted                  0.4175    0.5825
                 Vehicle                 0.4150    0.5850
                 Waste                   0.0000    1.0000
                 commute                 0.8350    0.1650

In [8]:
# Aggregate across categories, but keep split between facilities
facility_agg = category_agg.groupby('Facility').mean()
facility_agg

,Data Coverage,Data Gap
Facility,,
364-Paris Office,0.125000,0.875000
89-Headquarters,0.625312,0.374688


# Calculate Primary Data shares
The Data Coverage and Gap numbers only show whether or not data has been entered. They are not distinguishing between primary data or whether national averages are used. This is what we'll calculate in the next step. 

In [9]:
# The Primary Data share is only applicable for Business Travel, Energy, and Employee Commute.
# We do not provide national averages for Hosted or Cloud Servers, the Vehicle Fleet or Waste at the moment. 

data_shares = data[data['input_type'].str.contains('Business travel | Energy | commute')]
categories2 = ['Business travel', 'Energy', 'commute']

In [21]:
# We are again interested in the primary data shares per quarter and category. 

quarter_r2 = []
facility_r2 = []
category_r2 = []
data_coverage2 = []
share = []
data_gap2 = []

counter_t = 1

for t in timeframe: 
    for c in categories2:
        for f in facilities:
            # The Paris Office was opened in September 2021 only, so only Q4 is relevant (in September it was Laurent only)
            if (counter_t < 4) and (f == "364-Paris Office"):
                break
            else: 
                basis = data[data['input_type'].str.contains(c)]
                basis = basis[basis['facility'].str.contains(f)]
                basis = basis[basis['date(year-month)'].str.contains(t)]
                
                helper = basis.value.notnull().groupby([basis['date(year-month)']]).sum().astype(int).reset_index(name='count')
                coverage = round(100*(helper['count'] > 0).sum() / len(helper))
                
                quarter_r2.append(t)
                facility_r2.append(f)
                category_r2.append(c)
                data_coverage2.append(coverage)
                data_gap2.append(100-coverage)
                # up until here, the calculations were the same as above. Now we'll take it one step further
                # to calculate the primary data share if the coverage is greater than 0
                
                if coverage > 0: 
                    basis2 = basis[basis['input_type'].str.contains('national average')]
                    helper2 = basis2.value.notnull().groupby([basis['date(year-month)']]).sum().astype(int).reset_index(name='count')
                    primary = round(100*(helper2['count']).sum() / len(basis2))
                    share.append(primary)
                else: 
                    share.append("N/A")

primary_share = []
for a, b in zip(data_coverage2, share):
    primary_share.append(round(((a - b) / a)*100))
    
    
result_2 = pd.DataFrame(list(zip(quarter_r2, facility_r2, category_r2, data_gap2, primary_share)), columns = ['Quarter', 'Facility', 'Category', 'Data Gap in %', 'Primary Data Share in %'])

# optionally, save result of data gaps and data coverages to CSV file
#result_2.to_csv("Primary Data shares Plan A 2021.csv")

result_2

,Quarter,Facility,Category,Data Gap in %,Primary Data Share in %
0,-01|-02|-03,89-Headquarters,Business travel,33,51
1,-01|-02|-03,89-Headquarters,Energy,0,67
2,-01|-02|-03,89-Headquarters,commute,33,100
3,-04|-05|-06,89-Headquarters,Business travel,0,100
4,-04|-05|-06,89-Headquarters,Energy,0,100
5,-04|-05|-06,89-Headquarters,commute,0,100
6,-07|-08|-09,89-Headquarters,Business travel,0,100
7,-07|-08|-09,89-Headquarters,Energy,0,100
8,-07|-08|-09,89-Headquarters,commute,0,100
9,-10|-11|-12,89-Headquarters,Business travel,0,33


In [22]:
presentation = result_2.copy()
presentation = presentation.drop(columns=['Facility'])
presentation

,Quarter,Category,Data Gap in %,Primary Data Share in %
0,-01|-02|-03,Business travel,33,51
1,-01|-02|-03,Energy,0,67
2,-01|-02|-03,commute,33,100
3,-04|-05|-06,Business travel,0,100
4,-04|-05|-06,Energy,0,100
5,-04|-05|-06,commute,0,100
6,-07|-08|-09,Business travel,0,100
7,-07|-08|-09,Energy,0,100
8,-07|-08|-09,commute,0,100
9,-10|-11|-12,Business travel,0,33


# Backup - some test calculations. Please ignore.

In [35]:
time = []
category = []
facility = []
share = []

for m in months: 
    for c in categories:
        for f in facilities2: 
            
            time.append(m)
            category.append(c)
            facility.append(f)
            
            print (str (m) + " " + str(c)+ " " + str(f))
            basis = data[data['input_type'].str.contains(c)]
            basis = basis[basis['facility'].str.contains(f)]
            basis = basis[basis['date(year-month)'].str.contains(m)]
            
            helper = basis.value.notnull().groupby([basis['date(year-month)']]).sum().astype(int).reset_index(name='count')
            
            coverage = round(100*(helper['count']!=0).sum() / len(helper))
            # if helper count = 0, then print "no data" 
            if (coverage < 1) and (str(c) != 'Facility'):
                print("Percentage of average data: no data")
                share.append('no data')
            elif (str(c) != 'Hosted') and (str(c) != 'Cloud') and (str(c) != 'Vehicle')  and (str(c) != 'Facility'): 
                basis2 = basis[basis['input_type'].str.contains('national average')]
                #print(basis2)
                helper2 = basis2.value.notnull().groupby([basis['date(year-month)']]).sum().astype(int).reset_index(name='count')
                #print(helper2)
                coverage2 = round(100*(helper2['count']).sum() / len(basis2))
                print("Percentage of average data: " + str(coverage2) + " %" )
                share.append(coverage2)
            elif (str(c) == 'Hosted') or (str(c) == 'Cloud'): 
                basis2 = basis[basis['input_type'].str.contains('no data')]
                #print(basis2)
                helper2 = basis2.value.notnull().groupby([basis['date(year-month)']]).sum().astype(int).reset_index(name='count')
                #print(helper2)
                coverage2 = round(100*(helper2['count']).sum() / len(basis2))
                print("Percentage of average data: " + str(coverage2) + " %" )
                share.append(coverage2)
            else: 
                share.append('no data')
            print()

2021-01 Business travel Headquarters
Percentage of average data: no data

2021-01 Business travel Paris
Percentage of average data: no data

2021-01 Energy Headquarters
Percentage of average data: 0 %

2021-01 Energy Paris
Percentage of average data: no data

2021-01 commute Headquarters
Percentage of average data: no data

2021-01 commute Paris
Percentage of average data: no data

2021-01 Facility Headquarters

2021-01 Facility Paris

2021-01 Hosted Headquarters
Percentage of average data: no data

2021-01 Hosted Paris
Percentage of average data: no data

2021-01 Cloud Headquarters
Percentage of average data: no data

2021-01 Cloud Paris
Percentage of average data: no data

2021-01 Vehicle Headquarters
Percentage of average data: no data

2021-01 Vehicle Paris
Percentage of average data: no data

2021-01 Waste Headquarters
Percentage of average data: no data

2021-01 Waste Paris
Percentage of average data: no data

2021-02 Business travel Headquarters
Percentage of average data: 0 %



Percentage of average data: 0 %

2021-11 commute Paris
Percentage of average data: no data

2021-11 Facility Headquarters

2021-11 Facility Paris

2021-11 Hosted Headquarters
Percentage of average data: no data

2021-11 Hosted Paris
Percentage of average data: no data

2021-11 Cloud Headquarters
Percentage of average data: no data

2021-11 Cloud Paris
Percentage of average data: no data

2021-11 Vehicle Headquarters

2021-11 Vehicle Paris
Percentage of average data: no data

2021-11 Waste Headquarters
Percentage of average data: no data

2021-11 Waste Paris
Percentage of average data: no data

2021-12 Business travel Headquarters
Percentage of average data: 0 %

2021-12 Business travel Paris
Percentage of average data: no data

2021-12 Energy Headquarters
Percentage of average data: 0 %

2021-12 Energy Paris
Percentage of average data: no data

2021-12 commute Headquarters
Percentage of average data: no data

2021-12 commute Paris
Percentage of average data: no data

2021-12 Facility H

In [36]:
df = pd.DataFrame(list(zip(time, category, facility, share)), columns = ['Month', 'Category', 'Facility', '% of avg. data'])

In [37]:
df.to_csv("Average vs. accurate.csv")

In [38]:
df2 = df.replace(100, 1)
df2

,Month,Category,Facility,% of avg. data
0,2021-01,Business travel,Headquarters,no data
1,2021-01,Business travel,Paris,no data
2,2021-01,Energy,Headquarters,0
3,2021-01,Energy,Paris,no data
4,2021-01,commute,Headquarters,no data
...,...,...,...,...
187,2021-12,Cloud,Paris,no data
188,2021-12,Vehicle,Headquarters,no data
189,2021-12,Vehicle,Paris,no data
190,2021-12,Waste,Headquarters,no data


In [48]:
counter_t = 1
for t in timeframes: 
    for c in categories:
        counter = 2
        coverages = []
        for f in facilities2:
            # The Paris Office was opened in September 2021 only, so only Q4 is relevant (in September it was Laurent only)
            if (counter_t < 4) and (f == "Paris"):
                break
            else: 
                print (str (t) + "" + str(c)+ " " + str(f))
                basis = df2[df2['Category'].str.contains(c)]
                basis = basis[basis['Facility'].str.contains(f)]
                basis = basis[basis['Month'].str.contains(t)]
                
                #print(basis)
                
                no_data = (basis['% of avg. data'] == 'no data').sum()
                accurate = (basis['% of avg. data'] == 0).sum()
                average = (basis['% of avg. data'] == 1).sum()
                
                #print(no_data, accurate, average)
                
                if no_data == 3:
                    print("no data")
                else: 
                    print("Accurate of available data: " + str(accurate  / (accurate + average)*100) + " %")
            print()

-01|-02|-03Business travel Headquarters
Accurate of available data: 50.0 %

-01|-02|-03Energy Headquarters
Accurate of available data: 66.66666666666666 %

-01|-02|-03commute Headquarters
Accurate of available data: 100.0 %

-01|-02|-03Facility Headquarters
no data

-01|-02|-03Hosted Headquarters
no data

-01|-02|-03Cloud Headquarters
no data

-01|-02|-03Vehicle Headquarters
no data

-01|-02|-03Waste Headquarters
no data

-04|-05|-06Business travel Headquarters
Accurate of available data: 100.0 %

-04|-05|-06Energy Headquarters
Accurate of available data: 100.0 %

-04|-05|-06commute Headquarters
Accurate of available data: 100.0 %

-04|-05|-06Facility Headquarters
no data

-04|-05|-06Hosted Headquarters
Accurate of available data: 0.0 %

-04|-05|-06Cloud Headquarters
Accurate of available data: 0.0 %

-04|-05|-06Vehicle Headquarters
no data

-04|-05|-06Waste Headquarters
no data

-07|-08|-09Business travel Headquarters
Accurate of available data: 100.0 %

-07|-08|-09Energy Headquarters

In [183]:
counter_t = 1
for t in timeframes: 
    for c in categories:
        counter = 2
        coverages = []
        for f in facilities2:
            # The Paris Office was opened in September 2021 only, so only Q4 is relevant (in September it was Laurent only)
            if (counter_t < 4) and (f == "Paris"):
                break
            else: 
                print (str (t) + "" + str(c)+ " " + str(f))
                basis = data[data['input_type'].str.contains(c)]
                basis = basis[basis['facility'].str.contains(f)]
                basis = basis[basis['date(year-month)'].str.contains(t)]
                
                

                helper = basis.value.notnull().groupby([basis['date(year-month)']]).sum().astype(int).reset_index(name='count')
                
                coverage = round(100*(helper['count']!=0).sum() / len(helper))
                print("Coverage: " + str(coverage) + " %")
                
                #calculate national averages vs. actual data input
                if coverage > 0: 
                    basis2 = basis[basis['input_type'].str.contains("national average")]
                    acc_vs_nat = basis2.value.str.contains("yes").sum()

                    print("% reflecting accruate data: " + str(((len(basis2) - acc_vs_nat) / len(basis2))*100) + " %")

                coverages.append(coverage)
                counter = counter - 1
                if counter == 0:
                    coverage_all = mean(coverages)
                    print()
                    print("Coverage all: " + str(coverage_all) + " %")
                print()
                
                
    counter_t += 1
        

-01|-02|-03Business travel Headquarters
Coverage: 67 %
% reflecting accruate data: 66.66666666666666 %

-01|-02|-03Energy Headquarters
Coverage: 100 %
% reflecting accruate data: 66.66666666666666 %

-01|-02|-03commute Headquarters
Coverage: 67 %
% reflecting accruate data: 100.0 %

-01|-02|-03Facility Headquarters
Coverage: 100 %
% reflecting accruate data: nan %

-01|-02|-03Hosted Headquarters
Coverage: 0 %

-01|-02|-03Cloud Headquarters
Coverage: 0 %

-01|-02|-03Vehicle Headquarters
Coverage: 0 %

-01|-02|-03Waste Headquarters
Coverage: 0 %

-01|-02|-03heating Headquarters
Coverage: 100 %
% reflecting accruate data: 66.66666666666666 %

-01|-02|-03electricity Headquarters
Coverage: 100 %
% reflecting accruate data: 66.66666666666666 %

-04|-05|-06Business travel Headquarters
Coverage: 100 %
% reflecting accruate data: 100.0 %

-04|-05|-06Energy Headquarters
Coverage: 100 %
% reflecting accruate data: 100.0 %

-04|-05|-06commute Headquarters
Coverage: 100 %
% reflecting accruate data

/var/folders/z9/9tr36v3s5jxf7j2cs3v55y900000gn/T/ipykernel_79000/2329972210.py:28: RuntimeWarning: invalid value encountered in long_scalars
  print("% reflecting accruate data: " + str(((len(basis2) - acc_vs_nat) / len(basis2))*100) + " %")
